Версия моего питона: 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]

In [254]:
import pandas as pd ## Обновить pandas до последней версии
import numpy as np
import math
import os
import glob
import mysql.connector
import datetime
from datetime import date
from mysql.connector import errorcode
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

#### Укажите свой путь!  двойной знак \ обязательный!

In [255]:
path = 'C:\\Users\\User\\Documents\\Python Scripts\\IT\\Task_1'

#### Важно, чтобы  не было других эксель файлов директории

In [256]:
extension = 'xlsx'
os.chdir(path)
excel_files = [i for i in glob.glob('*.{}'.format(extension))]
applications = excel_files [0:64]
contracts = excel_files[64:]

# Пункт 1 (Загрузка данных и передача в БД)

### Таблица по заёмщикам

In [257]:
def get_features_for_one_person(application):
    column_names0 = []
    column_names1 = []
    for column in application:
        for row in range(1, 8, 2):
            column_names0.append(application[column][row])
        for row in range(12, 20, 2):
            column_names1.append(application[column][row])
    column_names = column_names0 + column_names1
    indicies_of_nan_values = []
    counter = 0
    for i in column_names:
        try:
            math.isnan(i)
            indicies_of_nan_values.append(counter)
            counter += 1
        except:
            counter += 1
    column_names = [x for i, x in enumerate(column_names) if i not in indicies_of_nan_values]
    values0 = []
    values1 = []
    for column in application:
        for row in range(2, 10, 2):
            values0.append(str(application[column][row])) ##str потому что, иначе рейзит ошибку с 3015ым годом в файле 11
        for row in range(13, 20, 2):
            try:
                values1.append(application[column][row])
            except:
                values1.append(np.nan)
    values = values0 + values1
    for i in values:
        try:
            math.isnan(i)
            indicies_of_nan_values.append(counter)
            counter += 1
        except:
            counter += 1
    values = [x for i, x in enumerate(values) if i not in indicies_of_nan_values]
    table = pd.DataFrame(columns = (column_names))
    table.loc[i] = values
    table['application_date'] = str(application[11][0])
    return table

In [258]:
applications_table = pd.DataFrame()
for i in applications:
        df = pd.read_excel(i, header = None, 
                           usecols = [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14],  skiprows = 1, nrows = 22)
        one_row = get_features_for_one_person(df)
        applications_table = applications_table.append(one_row)

In [259]:
#applications_table = applications_table.reset_index()
#applications_table.drop(['index'], axis = 1)
applications_table = applications_table.reset_index()

In [260]:
applications_table.keys()

Index(['index', 'Name', 'Identity Number', 'Street Address', 'City',
       'Issue Date', 'Mobile Number', 'Children', 'State', 'Home Number',
       'Family', 'Date of Birth', 'Email Address', 'Marital Status',
       'Zip Code', 'Gender', 'Employed By', 'Education', 'City',
       'Business Phone', 'Business Email', 'State', 'Position',
       'Date appointed', 'Income', 'Income Type', 'Housing',
       'Age of Car (if owned)', 'House ownership', 'State',
       'application_date'],
      dtype='object')

In [261]:
applications_table = applications_table.drop(['State', 'City', 'Name', 'Street Address', 'Mobile Number', 
                                              'Home Number', 'Email Address', 'Zip Code', 'Business Phone',
                                             'Business Email', 'Date appointed'], axis = 1)

In [262]:
applications_table['Identity Number'] = pd.to_numeric(applications_table['Identity Number'])
#applications_table['Children'] = pd.to_numeric(applications_table['Children'])
#applications_table['Family'] = pd.to_numeric(applications_table['Family'])
#applications_table['Income'] = pd.to_numeric(applications_table['Income'], downcast = 'integer')


In [263]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [264]:
ages = []
for i, value in enumerate(applications_table['Date of Birth']):
    if len(applications_table['Date of Birth'][i]) < 11:
        ages.append(calculate_age(datetime.datetime.strptime(applications_table['Date of Birth'][i], '%m.%d.%Y')))
    else:
        ages.append(calculate_age(datetime.datetime.strptime(applications_table['Date of Birth'][i], '%Y-%d-%m %H:%M:%S')))
        
applications_table['age'] = ages

In [265]:
for i, value in enumerate(applications_table['Issue Date']):
    if len(applications_table['Issue Date'][i]) < 11:
        applications_table['Issue Date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%m.%d.%Y'), "%Y/%m/%d")
    else:
        applications_table['Issue Date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%Y-%d-%m %H:%M:%S'), "%Y/%m/%d")

In [266]:
for i, value in enumerate(applications_table['application_date']):
    if len(applications_table['application_date'][i]) < 11:
        applications_table['application_date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%m.%d.%Y'), "%Y/%m/%d")
    else:
        applications_table['application_date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%Y-%d-%m %H:%M:%S'), "%Y/%m/%d")

In [267]:
applications_table = applications_table.replace(np.nan, 'NULL')
applications_table = applications_table.replace('nan', 'NULL')
#applications_table = applications_table.replace('NULL', np.nan)
#applications_table = applications_table.astype(object).where(pd.notnull(applications_table), None)
#applications_table

In [268]:
applications_table['Gender'] = applications_table['Gender'] == 'Female'

In [297]:
applications_table

,index,Identity Number,Issue Date,Children,Family,Date of Birth,Marital Status,Gender,Employed By,Education,Position,Income,Income Type,Housing,Age of Car (if owned),House ownership,application_date,age
0,NULL,100003,2013/08/29,0,2,07.22.1968,Married,True,School,Higher education,Core staff,270000,State servant,House / apartment,NULL,N,2014/06/16,50
1,NULL,100004,2010/01/28,0,1,1964-10-11 00:00:00,Single / not married,False,Government,Secondary / secondary special,Laborers,67500,Working,House / apartment,26,Y,2017/01/02,54
2,NULL,100006,2010/01/08,0,2,08.29.1964,Civil marriage,True,Business Entity Type 3,Secondary / secondary special,Laborers,135000,Working,House / apartment,NULL,Y,2016/09/10,54
3,NULL,100007,2008/03/18,0,1,1963-09-02 00:00:00,Single / not married,False,Religion,Secondary / secondary special,Core staff,121500,Working,House / apartment,NULL,Y,2017/09/05,55
4,NULL,100008,2013/11/24,0,2,10.27.1968,Married,False,Other,Secondary / secondary special,Laborers,99000,State servant,House / apartment,NULL,Y,2015/03/16,50
5,NULL,100009,2014/02/15,1,3,1978-05-02 00:00:00,Married,True,Business Entity Type 3,Higher education,Accountants,171000,Commercial associate,House / apartment,17,Y,2015/10/27,40
6,NULL,100010,2010/11/15,0,2,1965-11-10 00:00:00,Married,False,Other,Higher education,Managers,360000,State servant,House / apartment,8,Y,2017/05/21,53
7,NULL,100011,2005/11/22,0,2,06.26.1960,Married,True,XNA,Secondary / secondary special,<undefined>,112500,Pensioner,House / apartment,NULL,Y,2015/07/07,58
8,NULL,100012,2006/11/05,0,1,02.28.1978,Single / not married,False,Electricity,Secondary / secondary special,Laborers,135000,Working,House / apartment,NULL,Y,2017/10/10,40
9,NULL,100014,2013/09/18,1,3,10.26.1987,Married,True,Medicine,Higher education,Core staff,112500,Working,House / apartment,NULL,Y,2015/09/26,31


### Таблица по контрактам

In [270]:
def get_features_for_contract(contracts):
    column_names0 = []
    for column in contracts[[0, 1]]:
        for row in range(1, 8, 2):
            column_names0.append(contracts[column][row])
    indicies_of_nan_values = []
    counter = 0
    for i in column_names0:
        try:
            math.isnan(i)
            indicies_of_nan_values.append(counter)
            counter += 1
        except:
            counter += 1
    column_names0 = [x for i, x in enumerate(column_names0) if i not in indicies_of_nan_values]
    
    values0 = []
    for column in contracts[[0, 1]]:
        for row in range(2, 9, 2):
            values0.append(contracts[column][row]) 
    for i in values0:
        try:
            math.isnan(i)
            indicies_of_nan_values.append(counter)
            counter += 1
        except:
            counter += 1
    values0 = [x for i, x in enumerate(values0) if i not in indicies_of_nan_values]
    table = pd.DataFrame(columns = (column_names0))
    table.loc[i] = values0
    table['contract_date'] = str(contracts[2][0])
    return table

In [271]:
contracts_table = pd.DataFrame()
for i in contracts:
        df = pd.read_excel(i, header = None, usecols = [0, 4, 5],  skiprows = 1)
        one_row = get_features_for_contract(df)
        contracts_table = contracts_table.append(one_row)

In [272]:
contracts_table = contracts_table.reset_index()

In [273]:
for i, value in enumerate(contracts_table['contract_date']):
    if len(contracts_table['contract_date'][i]) < 11:
        contracts_table['contract_date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%m.%d.%Y'), "%Y/%m/%d")
    else:
        contracts_table['contract_date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%Y-%d-%m %H:%M:%S'), "%Y/%m/%d")

In [274]:
contracts_table = contracts_table.drop(['index', 'Borrower'], axis = 1)

In [275]:
contracts_table = contracts_table.replace(np.nan, 'NULL')
contracts_table = contracts_table.replace('nan', 'NULL')

In [276]:
contracts_table.head(5)

,Identity Number,Amount,Term (month),Contract Number,Type,Annuity,contract_date
0,100009,1560726.0,41,100022,Cash loans,41301.0,2015/11/02
1,100047,1193580.0,37,100052,Cash loans,35028.0,2016/08/30
2,100025,1132573.5,33,100068,Cash loans,37561.5,2018/12/30
3,100064,298728.0,22,100113,Cash loans,15381.0,2018/02/19
4,100023,544491.0,34,100122,Cash loans,17563.5,2016/07/31


## Загрузка в SQL

In [277]:
import mysql.connector
from mysql.connector import errorcode


try:
  cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')

except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
cursor = cnx.cursor()

In [278]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS applications1 
(id INT, 
issue_date DATE, 
children INT, 
family INT, 
age INT, 
marital_status VARCHAR(50), 
gender BOOLEAN,
employed_by VARCHAR(50), 
education VARCHAR(50),
position VARCHAR(50), 
income FLOAT,
income_type VARCHAR(50), 
housing VARCHAR(50),
age_of_car INT,
house_ownership VARCHAR(50),
application_date DATE,
PRIMARY KEY (id)) ENGINE=INNODB""")

cnx.commit()

In [279]:
cnx.close()

In [280]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

for i in range(0, len(applications_table['Identity Number'])):
    cursor.execute(""" INSERT INTO applications1 
    VALUES 
    (%d, '%s', %s, %s, %d, '%s', %s, '%s', '%s', '%s', %s, '%s', '%s', %s, '%s', '%s')""" % (
        applications_table['Identity Number'][i],
        applications_table['Issue Date'][i],
        applications_table['Children'][i], 
        applications_table['Family'][i], 
        applications_table['age'][i], 
        applications_table['Marital Status'][i], 
        applications_table['Gender'][i], 
        applications_table['Employed By'][i], 
        applications_table['Education'][i],
        applications_table['Position'][i], 
        applications_table['Income'][i], 
        applications_table['Income Type'][i], 
        applications_table['Housing'][i], 
        applications_table['Age of Car (if owned)'][i],
        applications_table['House ownership'][i],
        applications_table['application_date'][i]))

cnx.commit()

In [281]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS contracts 
(id INT, 
amount INT, 
term_month INT, 
contract_number INT, 
contract_type VARCHAR(50), 
annuity INT, 
contract_date DATE,
PRIMARY KEY (id),
FOREIGN KEY (id) REFERENCES applications1(id)) ENGINE=INNODB """)

cnx.commit()

In [282]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

for i in range(0, len(contracts_table['Identity Number'])):
    try:
        cursor.execute(""" INSERT INTO contracts
        VALUES 
        (%d, %d, %d, %d, '%s', %d, '%s')""" % (
            contracts_table['Identity Number'][i],
            contracts_table['Amount'][i],
            contracts_table['Term (month)'][i], 
            contracts_table['Contract Number'][i], 
            contracts_table['Type'][i], 
            contracts_table['Annuity'][i], 
            contracts_table['contract_date'][i]))
    except:
        print("Нет заявки с id = %d" %contracts_table['Identity Number'][i])

cnx.commit()

Нет заявки с id = 100002


# Пункт 2 (проверка корректности значений)

In [283]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

cursor.execute("SELECT COUNT(*) FROM applications1 WHERE children < 0")
if cursor.fetchall()[0][0] > 0:
    print("Найдены значения с children < 0")
    cursor.execute("UPDATE applications1 SET children=-1*(children) WHERE children < 0")
    print("Исправлены на положительные значения")
else:
    print("Не найдены значения с children < 0")


    
cursor.execute("SELECT COUNT(*) FROM applications1 WHERE family < 0")
if cursor.fetchall()[0][0] > 0:
    print("Найдены значения с family < 0")
    cursor.execute("UPDATE applications1 SET family=-1*(family) WHERE family < 0")
    print("Исправлены на положительные значения")
else:
    print("Не найдены значения с family < 0")
    
    

cursor.execute("SELECT COUNT(*) FROM applications1 WHERE income < 0")
if cursor.fetchall()[0][0] > 0:
    print("Найдены значения с income < 0")
    cursor.execute("UPDATE applications1 SET income=-1*(income) WHERE income < 0")
    print("Исправлены на положительные значения")
else:
    print("Не найдены значения с income < 0")
  


    
cnx.commit()

Не найдены значения с children < 0
Не найдены значения с family < 0
Не найдены значения с income < 0


In [284]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

print("Проверка на единство обозначений семейного положения")
cursor.execute("SELECT COUNT(marital_status), marital_status FROM applications1 GROUP BY marital_status")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

print("Проверка на единство обозначений занимаемой должности")
cursor.execute("SELECT COUNT(position), position FROM applications1 GROUP BY position")
for i in cursor.fetchall():
    print(i)  
cursor.execute("""UPDATE applications1 SET position = 'NULL' WHERE position = '<undefined>'""")
print("'<undefined>' заменено на NULL")
print("_" * 100)

print("Проверка на единство обозначений владения домом")
cursor.execute("SELECT COUNT(housing), housing FROM applications1 GROUP BY housing")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

cnx.commit()

Проверка на единство обозначений семейного положения
(39, 'Married')
(12, 'Single / not married')
(8, 'Civil marriage')
(3, 'Widow')
(1, 'Separated')
(1, 'NULL')
____________________________________________________________________________________________________
Проверка на единство обозначений занимаемой должности
(7, 'Core staff')
(17, 'Laborers')
(2, 'Accountants')
(6, 'Managers')
(14, '<undefined>')
(6, 'Drivers')
(6, 'Sales staff')
(1, 'Cleaning staff')
(1, 'Cooking staff')
(2, 'Private service staff')
(1, 'Medicine staff')
(1, 'Security staff')
'<undefined>' заменено на NULL
____________________________________________________________________________________________________
Проверка на единство обозначений владения домом
(58, 'House / apartment')
(2, 'Rented apartment')
(2, 'With parents')
(1, 'Municipal apartment')
(1, 'NULL')
____________________________________________________________________________________________________


In [285]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()


cursor.execute("""SELECT COUNT(*) FROM applications1 WHERE marital_status IN 
('Married', 'Civil marriage') AND children AND family <> (children + 2)""")
if cursor.fetchall()[0][0] > 0:
    print("Количество количество детей и родителей не совпадает с family")
    cursor.execute("""UPDATE applications1 SET family = children + 2 WHERE marital_status IN ('Married', 'Civil marriage')
    AND children AND family <> (children + 2)""")
    print("Сумма людей в семье исправлена")
else:
    print("Ошибки в количестве членов семьи не найдены")
    
cursor.execute("""SELECT COUNT(*) FROM applications1 WHERE marital_status IN 
('Single / not married', 'Widow', 'Separated') AND children AND family <> (children + 1)""")
if cursor.fetchall()[0][0] > 0:
    print("Количество количество детей и родителей не совпадает с family")
    cursor.execute("""UPDATE applications1 SET family = children + 1 WHERE marital_status IN 
    ('Single / not married', 'Widow', 'Separated') AND children AND family <> (children + 1)""")
    print("Сумма людей в семье исправлена")
else:
    print("Ошибки в количестве членов семьи не найдены")
    
cnx.commit()

Ошибки в количестве членов семьи не найдены
Ошибки в количестве членов семьи не найдены


In [286]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()


cursor.execute("SELECT COUNT(*) FROM contracts WHERE amount > annuity * term_month")
if cursor.fetchall()[0][0] > 0:
    print("Кредит выдан с ошибкой: тело кредита большн чем суммарные ежемесячные платежи")
else:
    print("Аннуитетные платежи рассчитаны верно")

Аннуитетные платежи рассчитаны верно


In [287]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

print("Проверка на единство обозначений типа контракта")
cursor.execute("SELECT COUNT(contract_type), contract_type FROM contracts GROUP BY contract_type")
for i in cursor.fetchall():
    print(i)   
print("_" * 100)

cnx.commit()

Проверка на единство обозначений типа контракта
(51, 'Cash loans')
(9, 'Revolving loans')
____________________________________________________________________________________________________


# Пункт 3 (Кодировка текстовых полей)

In [288]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

applications_df = pd.read_sql_query("SELECT * FROM applications1", cnx)
contracts_df = pd.read_sql_query("SELECT * FROM contracts", cnx)

In [289]:
cnx.close()

In [290]:
def encoding_columns_for_applications(column_name):
    le = preprocessing.LabelEncoder()
    le.fit(applications_df[column_name])
    numeric_values = le.transform(applications_df[column_name])
    
    ids = applications_df.id
    ids_and_values = list(zip(ids, numeric_values))
    
    cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
    cursor = cnx.cursor()
    
    cursor.execute("""ALTER TABLE applications1 ADD {0}_num int""".format(column_name))
    
    for i in ids_and_values:
        cursor.execute("""UPDATE applications1 SET {0}_num = %d WHERE id = %d """.format(column_name) %(i[1], i[0]))
      
    keys = le.inverse_transform(numeric_values)
    keys_and_values = dict(zip(keys, numeric_values))   
    
    cursor.execute("""CREATE TABLE IF NOT EXISTS dict_keys_applications
    (characteristic VARCHAR(50),
    key_num VARCHAR(50), 
    id_number INT)""")
    
    
    for i in keys_and_values:
        cursor.execute(""" INSERT INTO dict_keys_applications
        VALUES 
        ('%s', '%s', %d)""".format(column_name) % (column_name, i, keys_and_values[i]))
    
    cnx.commit()

In [291]:
encoding_columns = ['marital_status', 'employed_by', 'education', 'position', 'income_type', 'housing', 'house_ownership']

In [292]:
for i in encoding_columns:
    encoding_columns_for_applications(i)

In [293]:
def encoding_columns_for_contracts(column_name):
    le = preprocessing.LabelEncoder()
    le.fit(contracts_df[column_name])
    numeric_values = le.transform(contracts_df[column_name])
    
    ids = contracts_df.id
    ids_and_values = list(zip(ids, numeric_values))
    
    cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
    cursor = cnx.cursor()

    cursor.execute("""ALTER TABLE contracts ADD {0}_num int""".format(column_name))

    for i in ids_and_values:
        cursor.execute("""UPDATE contracts SET {0}_num = %d WHERE id = %d """.format(column_name) %(i[1], i[0]))
        
    keys = le.inverse_transform(numeric_values)
    keys_and_values = dict(zip(keys, numeric_values))   
    
    cursor.execute("""CREATE TABLE IF NOT EXISTS dict_keys_contracts
    ({0}_num VARCHAR(50), 
    id_number INT)""".format(column_name))

    
    for i in keys_and_values:
        cursor.execute(""" INSERT INTO dict_keys_contracts
        VALUES 
        ('%s', %d)""" % (i,
                       keys_and_values[i]))
    
    cnx.commit()
    cnx.close()

In [294]:
encoding_columns_for_contracts('contract_type')

In [295]:
cnx.close()

In [296]:
cnx = mysql.connector.connect(host = 'localhost', database = 'hw1_db', user = 'hw1_user')
cursor = cnx.cursor()

cursor.execute("ALTER TABLE contracts DROP contract_type")
cursor.execute("ALTER TABLE applications1 DROP marital_status")
cursor.execute("ALTER TABLE applications1 DROP employed_by")
cursor.execute("ALTER TABLE applications1 DROP education")
cursor.execute("ALTER TABLE applications1 DROP position")
cursor.execute("ALTER TABLE applications1 DROP income_type")
cursor.execute("ALTER TABLE applications1 DROP housing")
cursor.execute("ALTER TABLE applications1 DROP house_ownership")

cnx.commit()
cnx.close()